Анализ текстов на слова

In [2]:
import pandas as pd
import re
import nltk
import pymorphy3
from functools import lru_cache

In [329]:
dataset = pd.read_csv("/Users/chervonikov_alexey/Desktop/projects/Technopark_Spring_2025/diploma_project/Purify/purify_ml/data/raw/kartaslovsent.csv", sep=";")
dataset.head()

,term,tag,value,pstv,ngtv,neut,dunno,pstvNgtvDisagreementRatio
0,абажур,NEUT,0.08,0.185,0.037,0.580,0.198,0.00
1,аббатство,NEUT,0.10,0.192,0.038,0.578,0.192,0.00
2,аббревиатура,NEUT,0.08,0.196,0.000,0.630,0.174,0.00
3,абзац,NEUT,0.00,0.137,0.000,0.706,0.157,0.00
4,абиссинец,NEUT,0.28,0.151,0.113,0.245,0.491,0.19


In [330]:
# term_to_tag_dict = dict(zip(dataset['term'], (dataset['tag'], dataset['ngtv'])))
term_to_tag_dict = {row['term']: [row['tag'], row['value']] for _, row in dataset.iterrows()}

print("Размер словаря:", len(term_to_tag_dict))
print("Пример записи:", list(term_to_tag_dict.items())[0])

test_word = "сопляк"
if test_word in term_to_tag_dict:
    print(f"Слово '{test_word}' имеет метку: {term_to_tag_dict[test_word]}")
else:
    print(f"Слово '{test_word}' отсутствует в словаре")

Размер словаря: 46127
Пример записи: ('абажур', ['NEUT', 0.08])
Слово 'сопляк' имеет метку: ['NGTV', -1.0]


In [331]:
tag, prob = term_to_tag_dict.get("кончить", ["NEUT", 0])
print(tag, prob)
if tag == 'NGTV' and prob < -0.95:
    tag = "NGTV"
else:
	tag = "NEUT"

print(tag)

NEUT 0.31
NEUT


In [13]:
lemmatizer = pymorphy3.MorphAnalyzer()
lemma = lemmatizer.parse("бордель")[0].normal_form
print(lemma)

бордель


Регулярка на мат

In [179]:
mat_regex = r"""(?iux)(?<![а-яё])(?:
(?:(?:у|[нз]а|(?:хитро|не)?вз?[ыьъ]|с[ьъ]|(?:и|ра)[зс]ъ?|(?:о[тб]|п[оа]д)[ьъ]?|(?:\S(?=[а-яё]))+?[оаеи-])-?)?(?:
  [её](?:б(?!о[рй]|рач)|п[уа](?:ц|тс))|
  и[пб][ае][тцд][ьъ]
).*?|

(?:(?:н[иеа]|(?:ра|и)[зс]|[зд]?[ао](?:т|дн[оа])?|с(?:м[еи])?|а[пб]ч|в[ъы]?|пр[еи])-?)?ху(?:[яйиеёю]|л+и(?!ган)).*?|

бл(?:[эя]|еа?)(?:[дт][ьъ]?)?|

\S*?(?:
  п(?:
    [иеё]зд|
    ид[аое]?р|
    ед(?:р(?!о)|[аое]р|ик)|
    охую
  )|
  бля(?:[дбц]|тс)|
  [ое]ху[яйиеё]|
  хуйн
).*?|

(?:о[тб]?|про|на|вы)?м(?:
  анд(?:[ауеыи](?:л(?:и[сзщ])?[ауеиы])?|ой|[ао]в.*?|юк(?:ов|[ауи])?|е[нт]ь|ища)|
  уд(?:[яаиое].+?|е?н(?:[ьюия]|ей))|
  [ао]л[ао]ф[ьъ](?:[яиюе]|[еёо]й)
)|

елд[ауые].*?|
ля[тд]ь|
(?:[нз]а|по)х
)(?![а-яё])"""

PRONOUNS = ['я', 'ты', 'вы', 'он', 'она', 'оно', 'мы', 'они', 'вас', 'нас', 'их', 'его', 'её']
stopword_set = set(nltk.corpus.stopwords.words('russian'))
stopword_set = stopword_set.union({'это', 'который', 'весь', 'наш', 'свой', 'ещё', 'её', 'ваш', 'также', 'итак'})

Идем в лемматизацию

In [ ]:
lemmatizer = pymorphy3.MorphAnalyzer()
lemma = lemmatizer.parse(word)[0].normal_form

@lru_cache(maxsize=10000)
def get_lemma(word):
    return lemmatizer.parse(word)[0].normal_form

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = ' '.join(text.split())
    return text.lower()

def is_pronoun_or_stopword(word):
    return word in PRONOUNS or word in stopword_set

def split_compound_words(word):
    return re.findall(r'[А-Яа-яё]+', word)

word_cache = {}

def get_negative_words(text):
    cleaned_text = clean_text(text)
    print(cleaned_text)
    negative_words = set()
    mat_words = list(re.findall(mat_regex, cleaned_text, re.VERBOSE))
    mat_words = set(split_compound_words("_".join(mat_words)))
    words = split_compound_words(text)
    print(words)
    for word in words:
        if is_pronoun_or_stopword(word):
            continue

        if word in word_cache:
            if word_cache[word] == 'NGTV':
                negative_words.add(word)
            continue

        if word in mat_words:
            word_cache[word] = "NGTV"
            negative_words.add(word)
            continue

        if word == "сука":
            word_cache[word] = "NGTV"
            negative_words.add(word)
            continue

        lemma = get_lemma(word)
        
        if is_pronoun_or_stopword(lemma):
            continue
        
        if lemma in term_to_tag_dict:
            tag = term_to_tag_dict[lemma]
            word_cache[word] = tag
            if tag == 'NGTV':
                negative_words.add(word)
        else:
            word_cache[word] = 'NEUT'
    
    return list(negative_words)


Пример работы

In [182]:
text = '''Который год - в стране царствует смута и развал, властвует произвол финансово- чиновничьей олигархии.
Который год мы ожидаем обещанного благополучия и процветания, получая взамен безудержный рост цен, неплатежи по зарплатам и социальным пособиям, межнациональные войны и конфликты, бандитизм и коррупцию.
Довольно слушать бесконечные обещания и заверения чиновников, терпеть унижения и издевательства обнаглевших "реформаторов".
На попытку Ельцина, с помощью отставки правительства, уйти от ответственности за содеянное - ответим решительным: Ельцина - в отставку !
На попытку Ельцина путем политических рокировок продлить агонию ненавистного антинародного режима - ответим : НЕТ - антинародному курсу !На угрозы президента распустить Государственную Думу, выступающую за изменение курса "реформ ", заявим: Руки прочь от Государственной Думы! Даешь Правительство народного доверия!'''
negative_words = get_negative_words(text)
print("Негативные слова:", negative_words)

который год - в стране царствует смута и развал, властвует произвол финансово- чиновничьей олигархии. который год мы ожидаем обещанного благополучия и процветания, получая взамен безудержный рост цен, неплатежи по зарплатам и социальным пособиям, межнациональные войны и конфликты, бандитизм и коррупцию. довольно слушать бесконечные обещания и заверения чиновников, терпеть унижения и издевательства обнаглевших "реформаторов". на попытку ельцина, с помощью отставки правительства, уйти от ответственности за содеянное - ответим решительным: ельцина - в отставку ! на попытку ельцина путем политических рокировок продлить агонию ненавистного антинародного режима - ответим : нет - антинародному курсу !на угрозы президента распустить государственную думу, выступающую за изменение курса "реформ ", заявим: руки прочь от государственной думы! даешь правительство народного доверия!
['Который', 'год', 'в', 'стране', 'царствует', 'смута', 'и', 'развал', 'властвует', 'произвол', 'финансово', 'чиновнич

Исправление ошибок

In [291]:
errors_dataset = pd.read_csv("/Users/chervonikov_alexey/Desktop/projects/Technopark_Spring_2025/diploma_project/Purify/purify_ml/data/raw/orfo_and_typos.L1_5+PHON.csv", sep = ";").iloc[1:, :]
# errors_dataset['weight'] = errors_dataset['weight'].apply(lambda x: '{0:.15f}'.format(float(x)))
errors_dataset.head()

,CORRECT,MISTAKE,WEIGHT
1,болота,балото,0.2652
2,болота,боллото,0.0909
3,болота,болотоэ,0.0909
4,болото,палатаа,0.5000
5,болото,болотл,0.3333


In [ ]:
# string_weights = errors_dataset[~errors_dataset['weight'].apply(lambda x: str(x).replace('.', '').isdigit())]
# string_weights

In [16]:
errors_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90971 entries, 1 to 90971
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CORRECT  90971 non-null  object 
 1   MISTAKE  90971 non-null  object 
 2   WEIGHT   90971 non-null  float64
dtypes: float64(1), object(2)
memory usage: 2.1+ MB


In [293]:
import pandas as pd
from collections import defaultdict
import re
from Levenshtein import distance as levenshtein_distance
from functools import lru_cache

class SpellChecker:
    def __init__(self, dataset_path, max_distance=2):
        self.correct_words = set()
        self.error_to_correct = defaultdict(list)
        self.max_distance = max_distance
        
        df = pd.read_csv(dataset_path, sep=';', header=None, 
                        names=['correct', 'error', 'weight']).iloc[1:, :]
        
        for _, row in df.iterrows():
            correct = row['correct'].strip().lower()
            error = row['error'].strip().lower()
            weight = float(row['weight'])
            
            self.correct_words.add(correct)
            self.error_to_correct[error].append((correct, weight))
        
        for error in self.error_to_correct:
            self.error_to_correct[error].sort(key=lambda x: x[1], reverse=True)
        
        self.all_known_words = list(self.correct_words) + list(self.error_to_correct.keys())
    
    @lru_cache(maxsize=10000)
    def find_closest_word(self, word):
        if not word:
            return None
            
        word = word.lower()
        
        if word in self.correct_words:
            return word
        if word in self.error_to_correct:
            return self.error_to_correct[word][0][0]
        
        min_distance = float('inf')
        closest_word = None
        
        for known_word in self.all_known_words:
            current_distance = levenshtein_distance(word, known_word)
            if current_distance < min_distance and current_distance <= self.max_distance:
                min_distance = current_distance
                closest_word = known_word
        
        return closest_word
    
    def correct_spelling(self, word):
        word = word.lower().strip()
        
        if hasattr(self, '_spelling_cache') and word in self._spelling_cache:
            return self._spelling_cache[word]
        
        if word in self.correct_words:
            return word
        
        if word in self.error_to_correct:
            correction = self.error_to_correct[word][0][0]
            if not hasattr(self, '_spelling_cache'):
                self._spelling_cache = {}
            self._spelling_cache[word] = correction
            return correction
        
        import time
        start = time.time()
        closest_word = self.find_closest_word(word)
        end = time.time()
        print(f"{end - start}")
        if closest_word:
            if closest_word in self.error_to_correct:
                correction = self.error_to_correct[closest_word][0][0]
            else:
                correction = closest_word
            
            if not hasattr(self, '_spelling_cache'):
                self._spelling_cache = {}
            self._spelling_cache[word] = correction
            return correction
        
        return None
    
    def correct_text(self, text):
        tokens = re.findall(r"(\w+|\W+)", text)
        corrected_tokens = []
        for token in tokens:
            if token.strip() and token[0].isalpha():  
                correction = self.correct_spelling(token)
                if correction is not None:
                    if token[0].isupper():
                        correction = correction.capitalize()
                    corrected_tokens.append(correction)
                else:
                    corrected_tokens.append(token)
            else:
                corrected_tokens.append(token)
        
        return ''.join(corrected_tokens)

checker = SpellChecker("/Users/chervonikov_alexey/Desktop/projects/Technopark_Spring_2025/diploma_project/Purify/purify_ml/data/raw/orfo_and_typos.L1_5+PHON.csv")
print(checker.correct_spelling("оборзели"))
text = "Это тествый текст с ашибками и опечатками некрасыво."
print(checker.correct_text(text))

0.02596902847290039
None
0.021033048629760742
0.022637128829956055
0.022774934768676758
0.02228713035583496
Это тесты текст с ашибками и опечатками некрасиво.


In [294]:
print(checker.correct_spelling("конченный"))

0.05901384353637695
оконченный


In [154]:
print(checker.correct_spelling("туптй"))  
print(checker.correct_spelling("несуществующееслово"))  
text = "Ты тупкя!"

corrected = checker.correct_text(text)
print(corrected)  

None
None
Ты тупкя!


In [2]:
import flask_cors
print(flask_cors.__version__)

5.0.1


In [3]:
mat_regex = r"""(?iux)(?<![а-яё])(?:
(?:(?:у|[нз]а|(?:хитро|не)?вз?[ыьъ]|с[ьъ]|(?:и|ра)[зс]ъ?|(?:о[тб]|п[оа]д)[ьъ]?|(?:\S(?=[а-яё]))+?[оаеи-])?[- ]*)?(?:
  [её](?:б(?!о[рй]|рач)|[- ]*п[уа](?:[- ]*ц|[- ]*тс))|
  [ий][- ]*[пб][- ]*[ае][- ]*[тцд][- ]*[ьъ]
).*?|

(?:(?:н[ийеа]|(?:ра|и)[зс]|[зд]?[ао](?:т|дн[оа])?|с(?:м[еий])?|а[пб]ч|в[ъы]?|пр[еий])[- ]*)?ху(?:[яйиеёю]|л+[ий](?!ган)).*?|

бл(?:[эя]|еа?)(?:[дт][ьъ]?)?|

\S*?(?:
  п(?:
    [иеё]+[- ]*[зс]+[- ]*д|
    [ий][- ]*д[аое]?[- ]*р|
    е[- ]*д(?:[- ]*р(?!о)|[аое][- ]*р|[ий][- ]*к)|
    о[- ]*х[- ]*у[- ]*ю
  )|
  бл[- ]*я(?:[дбц]|тс)|
  [ое][- ]*х[- ]*у[яйиеё]|
  х[- ]*у[- ]*[йи][- ]*н
).*?|

(?:о[тб]?|про|на|вы)?м(?:
  а[- ]*н[- ]*д(?:[ауеыи](?:л(?:[ий][сзщ])?[ауеиы])?|ой|[ао]в.*?|юк(?:ов|[ауи])?|е[нт]ь|[ий]ща)|
  у[- ]*д(?:[яаиое].+?|е?н(?:[ьюия]|ей))|
  [ао][- ]*л[ао][- ]*ф[ьъ](?:[яиюе]|[еёо][йи])
)|

е[- ]*л[- ]*д[ауые].*?|
л[- ]*я[тд][- ]*ь|
(?:[нз]а|по)х|

(?:г[- ]*а[- ]*н[- ]*д[- ]*о[- ]*н[а-яё]*|м[- ]*о[- ]*з[- ]*г[- ]*о[- ]*[её][- ]*б[а-яё]*|х[- ]*у[- ]*[йи][- ]*л[- ]*о[а-яё]*|з[- ]*а[- ]*л[- ]*у[- ]*п[- ]*а[а-яё]*|
п(?:[- ]*[ий][- ]*з[- ]*д[её]?ц|[- ]*з[- ]*д[- ]*ц)|в[- ]*ы[- ]*п[- ]*[ий][- ]*з[- ]*д[а-яё]*|м[- ]*у[- ]*д[- ]*а[- ]*к[а-яё]*|е[- ]*б[- ]*л[- ]*а[а-яё]*|
ш[- ]*а[- ]*л[- ]*а[- ]*в[а-яё]*|д[- ]*е[- ]*р[- ]*ь[- ]*м[- ]*о[- ]*е[- ]*д[а-яё]*|с[- ]*к[- ]*о[- ]*т[- ]*о[- ]*б[- ]*л[- ]*ю[- ]*д[а-яё]*|
ч[- ]*м[- ]*о[- ]*ш[- ]*н[- ]*[ий][- ]*к[а-яё]*|п[- ]*р[- ]*о[- ]*с[- ]*т[- ]*о[- ]*ф[- ]*[ий][- ]*л[а-яё]*)|

# 1. Общие оскорбления
(?:у[- ]*р[- ]*о[- ]*д[а-яё]*|д[- ]*е[- ]*б[- ]*[ий][лв][а-яё]*|д[- ]*а[- ]*у[- ]*н[а-яё]*|к[- ]*р[- ]*е[- ]*т[- ]*[ий][- ]*н[а-яё]*|[ий][- ]*д[ий][ао][- ]*т[а-яё]*|д[- ]*о[- ]*л[- ]*б[- ]*о[ёе][- ]*б[а-яё]*|
п[- ]*р[- ]*[ий][- ]*д[- ]*у[- ]*р[а-яё]*|о[- ]*т[- ]*м[- ]*о[- ]*р[- ]*о[- ]*з[а-яё]*|г[- ]*н[- ]*[ий][- ]*д[а-яё]*|п[- ]*о[- ]*д[- ]*о[- ]*н[а-яё]*|о[- ]*т[- ]*б[- ]*р[- ]*о[- ]*с[а-яё]*|м[- ]*е[- ]*р[- ]*з[- ]*а[- ]*в[а-яё]*|
н[- ]*е[- ]*г[- ]*о[- ]*д[- ]*я[йя][а-яё]*|п[- ]*о[- ]*д[- ]*л[- ]*е[- ]*ц[а-яё]*|[ий][- ]*с[- ]*ч[- ]*а[- ]*д[- ]*[ий][а-яё]*|ч[- ]*м[оа][а-яё]*|л[- ]*о[- ]*х[а-яё]*|п[- ]*а[- ]*д[- ]*л[ао][йея]|п[- ]*а[- ]*д[- ]*л[- ]*ю[кг][а-яё]*)|

# 2. Животные-оскорбления
(?:с[- ]*к[- ]*о[- ]*т[- ]*[ий][- ]*н[а-яё]*|п[- ]*с[- ]*[ий][- ]*н[а-яё]*|с[- ]*в[- ]*[ий][- ]*н[- ]*ь[яи][а-яё]*|о[- ]*в[- ]*ц[аеы][а-яё]*|к[- ]*о[- ]*з[ёе][- ]*л[а-яё]*|б[- ]*з[- ]*д[а-яё]*)|

# 3. Сексуально-оскорбительные и разврат
(?:ш[- ]*л[- ]*ю[- ]*х[а-яё]*|п[- ]*р[- ]*о[- ]*с[- ]*т[- ]*и[- ]*т[- ]*у[- ]*т[- ]*к[а-яё]*|б[- ]*л[- ]*у[- ]*д[- ]*н[- ]*[ий][- ]*ц[а-яё]*|р[- ]*а[- ]*з[- ]*в[- ]*р[- ]*а[- ]*т[- ]*н[- ]*[ий][- ]*к[а-яё]*|п[- ]*о[- ]*х[- ]*о[- ]*т[а-яё]*|б[- ]*л[- ]*я[- ]*д[а-яё]*|
с[- ]*о[- ]*д[- ]*о[- ]*м[а-яё]*|г[- ]*о[- ]*м[- ]*о[- ]*с[- ]*е[- ]*к[а-яё]*|п[- ]*е[- ]*д[- ]*о[- ]*ф[- ]*[ий][- ]*л[а-яё]*|[ий][- ]*з[- ]*в[- ]*р[- ]*а[- ]*щ[- ]*е[- ]*н[а-яё]*|п[- ]*р[ийо][- ]*с[- ]*т[ийо][- ]*т[а-яё]*|с[- ]*е[- ]*к[- ]*с[- ]*о[- ]*т[а-яё]*|
т[- ]*р[- ]*а[- ]*х[а-яё]*|е[- ]*б[а-яё]*|п[- ]*о[- ]*р[- ]*н[- ]*о[а-яё]*|в[- ]*а[- ]*г[- ]*[ий][- ]*н[а-яё]*|ч[- ]*л[- ]*е[- ]*м[а-яё]*|с[- ]*о[- ]*с[аи][а-яё]*|ж[- ]*[оа][- ]*п[а-яё]*)

# 4. Алкоголь и наркотики
(?:а[- ]*л[- ]*к[- ]*а[- ]*ш[а-яё]*|п[- ]*ь[- ]*я[- ]*н[- ]*ь[а-яё]*|б[- ]*у[- ]*х[- ]*л[ао][а-яё]*|н[- ]*а[- ]*р[- ]*к[- ]*о[- ]*м[- ]*а[- ]*н[а-яё]*|д[- ]*о[- ]*з[а-яё]*|г[- ]*е[- ]*р[- ]*о[- ]*[ий][- ]*н[а-яё]*|
к[- ]*о[- ]*к[- ]*с[а-яё]*|а[- ]*н[- ]*а[- ]*ш[а-яё]*|с[- ]*п[- ]*а[- ]*[ий][- ]*с[а-яё]*|ш[- ]*[ий][- ]*р[а-яё]*|т[- ]*о[- ]*р[- ]*ч[а-яё]*|д[- ]*р[- ]*ю[- ]*ч[а-яё]*|к[- ]*о[- ]*л[ёе][- ]*ц[а-яё]*|
м[- ]*а[- ]*р[- ]*[ий][- ]*х[- ]*у[- ]*а[- ]*н[а-яё]*|г[- ]*а[- ]*ш[а-яё]*|п[ао][- ]*р[- ]*н[а-яё]*|з[- ]*а[- ]*к[- ]*л[- ]*а[- ]*д[а-яё]*|д[- ]*р[- ]*я[- ]*н[- ]*ь[а-яё]*)|

# 5. Социальные оскорбления
(?:б[- ]*о[- ]*м[- ]*ж[а-яё]*|п[- ]*а[- ]*р[- ]*а[- ]*з[- ]*[ий][- ]*т[а-яё]*|ш[- ]*а[- ]*р[- ]*о[- ]*м[- ]*ы[- ]*ж[- ]*н[- ]*[ий][- ]*к[а-яё]*|у[- ]*б[- ]*л[- ]*ю[- ]*д[а-яё]*|в[- ]*ы[- ]*б[- ]*л[- ]*я[- ]*д[а-яё]*)|

# 6. "Грязные" слова
(?:[гж][- ]*а[- ]*[вф][- ]*н[аоы]|г[ао][- ]*в[- ]*н[аоу]|г[- ]*о[- ]*в[- ]*е[- ]*н[а-яё]*|г[- ]*а[- ]*в[- ]*н[- ]*ю[кч]|д[- ]*е[- ]*р[- ]*ь[- ]*м[аоуы]|д[- ]*е[- ]*р[ьъ][- ]*м[аоу]|д[еи][- ]*р[- ]*ь[- ]*м[а-яё]*)|

# 7. Оскорбления женского рода (обновлено)
(?:с[ую][- ]*[кч][аиуео]|с[- ]*у[- ]*ч[аио][йея]|с[- ]*у[- ]*ч[- ]*к[аиу]|с[- ]*у[- ]*ч[- ]*ь[еяи]|с[- ]*т[- ]*е[- ]*р[- ]*в[а-яё]*|х[- ]*р[- ]*ы[- ]*ч[а-яё]*|
ш[- ]*м[- ]*а[- ]*р[а-яё]*|ш[- ]*в[- ]*а[- ]*л[- ]*ь|ш[- ]*л[- ]*ю[- ]*х[а-яё]*|б[- ]*л[- ]*я[- ]*д[а-яё]*|п[- ]*о[- ]*т[- ]*а[- ]*с[- ]*к[а-яё]*|п[- ]*р[оа][- ]*л[её][- ]*т[- ]*к[а-яё]*)|

# 8. Другие
(?:х[- ]*а[- ]*м[а-яё]*|х[- ]*а[- ]*м[- ]*л[ао][а-яё]*|м[- ]*р[- ]*а[- ]*з[аоуи]|м[- ]*р[- ]*а[- ]*з[- ]*[ий]|м[- ]*р[- ]*а[- ]*з[- ]*о[- ]*т[а-яё]*|а[- ]*у[- ]*т[- ]*[ий][- ]*с[- ]*т[а-яё]*|д[- ]*е[- ]*г[- ]*е[- ]*н[- ]*е[- ]*р[- ]*а[- ]*т[а-яё]*)
)(?![а-яё])"""

is_mat = bool(re.search(mat_regex, "проститутка", flags=re.VERBOSE))
print(is_mat)

False


In [2]:
mat_regex = r"""(?iux)(?<![а-яё])(?:
# Исходные матерные паттерны (с поддержкой дефисов и пробелов)
(?:(?:у|[- ]*[нз][- ]*а|(?:х[- ]*и[- ]*т[- ]*р[- ]*о|н[- ]*е)?[- ]*в[- ]*з?[- ]*[ыьъ]|[- ]*с[- ]*[ьъ]|(?:и|р[- ]*а)[- ]*[зс][- ]*ъ?|(?:о[- ]*[тб]|п[- ]*[оа][- ]*д)[- ]*[ьъ]?|(?:\S(?=[а-яё]))+?[- ]*[оаеи][- ]*)[- ]*)?(?:
  [- ]*[её][- ]*(?:[- ]*б(?![- ]*о[- ]*[рй]|[- ]*р[- ]*а[- ]*ч)|[- ]*п[- ]*[уа][- ]*(?:[- ]*ц|[- ]*т[- ]*с))|
  [- ]*и[- ]*[пб][- ]*[ае][- ]*[тцд][- ]*[ьъ]
).*?|

(?:(?:н[- ]*[иеа]|(?:р[- ]*а|и)[- ]*[зс]|[- ]*[зд][- ]*?[ао][- ]*(?:т|д[- ]*н[- ]*[оа])?|[- ]*с[- ]*(?:м[- ]*[еи])?|[- ]*а[- ]*[пб][- ]*ч|[- ]*в[- ]*[ъы]?|[- ]*п[- ]*р[- ]*[еи])[- ]*)?х[- ]*у(?:[- ]*[яйиеёю]|[- ]*л+[- ]*и(?!г[- ]*а[- ]*н)).*?|

б[- ]*л(?:[- ]*[эя]|[- ]*е[- ]*а?)(?:[- ]*[дт][- ]*[ьъ]?)?|

\S*?(?:
  п[- ]*(?:
    (?:[- ]*[иеё][- ]*[зс][- ]*д|[- ]*з[- ]*д)[- ]*(?:[- ]*[еёц])? |  
    [- ]*[иеё][- ]*з[- ]*д|  
    [- ]*и[- ]*д[- ]*[аое][- ]*?р|
    [- ]*е[- ]*д(?:[- ]*р(?![- ]*о)|[- ]*[аое][- ]*р|[- ]*и[- ]*к)|
    [- ]*о[- ]*х[- ]*у[- ]*ю
  )|
  б[- ]*л[- ]*я(?:[- ]*[дбц]|[- ]*т[- ]*с)|
  [- ]*[ое][- ]*х[- ]*у[- ]*[яйиеё]|
  х[- ]*у[- ]*й[- ]*н
).*?|

(?:о[- ]*[тб]?|п[- ]*р[- ]*о|н[- ]*а|в[- ]*ы)?м[- ]*(?:
  а[- ]*н[- ]*д(?:[- ]*[ауеыий](?:[- ]*л(?:[- ]*и[- ]*[сзщ])?[- ]*[ауеийы])?|[- ]*о[- ]*й|[- ]*[ао][- ]*в.*?|[- ]*ю[- ]*к(?:[- ]*о[- ]*в|[- ]*[ауий])?|[- ]*е[- ]*[нт][- ]*ь|[- ]*и[- ]*щ[- ]*а)|
  у[- ]*д(?:[- ]*[яаийое].+?|[- ]*е?[- ]*н(?:[- ]*[ьюийя]|[- ]*е[- ]*й))|
  [- ]*[ао][- ]*л[- ]*[ао][- ]*ф[- ]*[ьъ](?:[- ]*[яийюе]|[- ]*[её][- ]*о[- ]*й)
)|

е[- ]*л[- ]*д[- ]*[ауые].*?|
л[- ]*я[- ]*[тд][- ]*ь|
(?:[- ]*[нз][- ]*а|п[- ]*о)[- ]*х|

# Грубые/оскорбительные слова (не мат)
# 1. Общие оскорбления
(?:у[- ]*р[- ]*о[- ]*д[- ]*[а-яё-]*|д[- ]*[эеи][- ]*б[- ]*и[- ]*[лчв][- ]*[а-яё-]*|д[- ]*а[- ]*у[- ]*н[- ]*[а-яё-]*|к[- ]*р[- ]*е[- ]*т[- ]*[ий][- ]*н[- ]*[а-яё-]*|[ий][- ]*д[- ]*[ио][- ]*[ао][- ]*т[- ]*[а-яё-]*|д[- ]*о[- ]*л[- ]*б[- ]*о[- ]*[ёе][- ]*б[- ]*[а-яё-]*|
п[- ]*р[- ]*[ий][- ]*д[- ]*у[- ]*р[- ]*[а-яё-]*|о[- ]*т[- ]*м[- ]*о[- ]*р[- ]*о[- ]*з[- ]*[а-яё-]*|г[- ]*н[- ]*и[- ]*д[- ]*[а-яё-]*|п[- ]*о[- ]*д[- ]*о[- ]*н[- ]*[а-яё-]*|о[- ]*т[- ]*б[- ]*р[- ]*о[- ]*с[- ]*[а-яё-]*|м[- ]*е[- ]*р[- ]*з[- ]*а[- ]*в[- ]*[а-яё-]*|
н[- ]*е[- ]*г[- ]*о[- ]*д[- ]*я[- ]*[йя][- ]*[а-яё-]*|п[- ]*о[- ]*д[- ]*л[- ]*е[- ]*ц[- ]*[а-яё-]*|[ий][- ]*с[- ]*ч[- ]*а[- ]*д[- ]*[ий][- ]*[а-яё-]*|ч[- ]*м[- ]*[оыа][- ]*[а-яё-]*|л[- ]*[оу][- ]*[хш][- ]*[а-яё-]*|
п[- ]*а[- ]*д[- ]*л[- ]*[уайея][- ]*|п[- ]*а[- ]*д[- ]*л[- ]*ю[- ]*[кг][- ]*[а-яё-]*|и[- ]*м[- ]*б[- ]*и[- ]*ц[- ]*и[- ]*л[- ]*[а-яё-]*|
д[- ]*у[- ]*р[- ]*[а-яё]*|м[- ]*у[- ]*д(?:[а-яё-]+[а-яё-]|[- ]*[^а-яё-])[а-яё-]*|[а-яё-]*х[- ]*е[- ]*р[- ]*[а-яё-]*)|

# 2. Животные-оскорбления
(?:с[- ]*к[- ]*о[- ]*т[- ]*[ий][- ]*н[- ]*[а-яё-]*|п[- ]*с[- ]*[ий][- ]*н[- ]*[а-яё-]*|с[- ]*в[- ]*[ий][- ]*н[- ]*ь[- ]*[яий][- ]*[а-яё-]*|о[- ]*в[- ]*ц[- ]*[аеы][- ]*[а-яё-]*|к[- ]*о[- ]*з[- ]*[ёе][- ]*л[- ]*[а-яё-]*|б[- ]*з[- ]*д[- ]*[а-яё-]*|
о[- ]*б[- ]*о[- ]*р[- ]*м[- ]*о[- ]*т[- ]*[а-яё-]*|б[- ]*а[- ]*р[- ]*а[- ]*н[- ]*[а-яё-]*|[оа][- ]*с[- ]*[её][- ]*л[- ]*|[оа][- ]*с[- ]*л[- ]*[ыуаоми]*|к[- ]*о[- ]*з[- ]*л[- ]*[а-яё-]*)|

# 3. Сексуально-оскорбительные и разврат
(?:ш[- ]*л[- ]*ю[- ]*х[- ]*[а-яё-]*|п[- ]*р[- ]*о[- ]*с[- ]*т[- ]*[ий][- ]*т[- ]*у[- ]*[а-яё-]*|б[- ]*л[- ]*у[- ]*д[- ]*н[- ]*[ий][- ]*ц[- ]*[а-яё-]*|р[- ]*а[- ]*з[- ]*в[- ]*р[- ]*а[- ]*т[- ]*н[- ]*[ий][- ]*к[- ]*[а-яё-]*|п[- ]*о[- ]*х[- ]*о[- ]*т[- ]*[а-яё-]*|б[- ]*л[- ]*я[- ]*д[- ]*[а-яё-]*|
с[- ]*о[- ]*д[- ]*о[- ]*м[- ]*[а-яё-]*|г[- ]*о[- ]*м[- ]*о[- ]*с[- ]*е[- ]*к[- ]*[а-яё-]*|п[- ]*е[- ]*д[- ]*о[- ]*ф[- ]*[ий][- ]*л[- ]*[а-яё-]*|и[- ]*з[- ]*в[- ]*р[- ]*а[- ]*щ[- ]*е[- ]*н[- ]*[а-яё-]*|п[- ]*р[- ]*[ио][- ]*с[- ]*т[- ]*[ио][- ]*т[- ]*[а-яё-]*|с[- ]*е[- ]*к[- ]*с[- ]*о[- ]*т[- ]*[а-яё-]*|
т[- ]*р[- ]*а[- ]*х[- ]*[а-яё]*|е[- ]*б[- ]*[а-яё-]*|п[- ]*о[- ]*р[- ]*н[- ]*о[- ]*[а-яё-]*|в[- ]*а[- ]*г[- ]*[ий][- ]*н[- ]*[а-яё-]*|ч[- ]*л[- ]*е[- ]*н[- ]*[а-яё-]*|[а-яё-]*с[- ]*[оа][- ]*с[- ]*[аиеёй][- ]*[а-яё-]*|[а-яё-]*ж[- ]*о[- ]*п[- ]*[а-яё-]*|б[- ]*л[- ]*у[- ]*д[- ]*[а-яё-]*|б[- ]*л[- ]*у[- ]*д[- ]*н[- ]*[ий][- ]*к[- ]*[а-яё-]*|
м[- ]*[ий][- ]*н[- ]*е[- ]*т[- ]*[а-яё-]*|г[- ]*[оа][- ]*н[- ]*д[- ]*о[- ]*н[- ]*[а-яё-]*|м[- ]*[ий][- ]*н[- ]*ь[- ]*е[- ]*т[- ]*|п[- ]*е[- ]*н[- ]*и[- ]*с[- ]*[а-яё-]*|[а-яё-]*з[- ]*а[- ]*л[- ]*у[- ]*п[- ]*[а-яё-]*)|

# 4. Алкоголь и наркотики
(?:а[- ]*л[- ]*к[- ]*а[- ]*ш[- ]*[а-яё-]*|п[- ]*ь[- ]*я[- ]*н[- ]*[а-яё-]*|б[- ]*у[- ]*х[- ]*л[- ]*[аио][- ]*[а-яё-]*|н[- ]*а[- ]*р[- ]*к[- ]*о[- ]*м[- ]*а[- ]*н[- ]*[а-яё-]*|д[- ]*о[- ]*з[- ]*[а-яё-]*|г[- ]*е[- ]*р[- ]*о[- ]*(?:[ий](?!ня)[- ]*н[- ]*[а-яё-]*|й[- ]*н[- ]*[а-яё-]*)|
к[- ]*о[- ]*к[- ]*с[- ]*[а-яё-]*|а[- ]*н[- ]*а[- ]*ш[- ]*[а-яё-]*|с[- ]*п[- ]*а[- ]*[ий][- ]*с[- ]*[а-яё-]*|ш[- ]*[ий][- ]*р[- ]*[а-яё-]*|т[- ]*о[- ]*р[- ]*ч[- ]*[а-яё-]*|д[- ]*р[- ]*ю[- ]*ч[- ]*[а-яё-]*|к[- ]*о[- ]*л[- ]*[ёе][- ]*ц[- ]*[а-яё-]*|
м[- ]*а[- ]*р[- ]*[ий][- ]*х[- ]*у[- ]*а[- ]*н[- ]*[а-яё-]*|г[- ]*а[- ]*ш[- ]*[а-яё-]*|п[- ]*[ао][- ]*р[- ]*н[- ]*[а-яё-]*|з[- ]*а[- ]*к[- ]*л[- ]*а[- ]*д[- ]*[а-яё-]*|д[- ]*р[- ]*я[- ]*н[- ]*ь[- ]*[а-яё-]*|
[а-яё-]*д[- ]*р[- ]*[ао][- ]*[чт][- ]*[а-яё-]*|н[- ]*а[- ]*р[- ]*к[- ]*о[- ]*[шт][- ]*[а-яё-]*)|
# 5. Социальные оскорбления
(?:б[- ]*о[- ]*м[- ]*ж[- ]*[а-яё-]*|п[- ]*а[- ]*р[- ]*а[- ]*з[- ]*[ий][- ]*т[- ]*[а-яё-]*|ш[- ]*а[- ]*р[- ]*о[- ]*м[- ]*ы[- ]*ж[- ]*н[- ]*[ий][- ]*к[- ]*[а-яё-]*|у[- ]*б[- ]*л[- ]*ю[- ]*д[- ]*[а-яё-]*|в[- ]*ы[- ]*б[- ]*л[- ]*я[- ]*д[- ]*[а-яё-]*|б[- ]*и[- ]*ч[- ]*[а-яё-]*)|

# 6. "Грязные" слова
(?:[гж][- ]*а[- ]*[вф][- ]*н[- ]*[аоы]|[а-яё-]*г[- ]*[ао][- ]*в[- ]*н[- ]*[а-яё-]*[- ]*|г[- ]*[оа][- ]*в[- ]*е[- ]*н[- ]*[а-яё-]*|г[- ]*[ао][- ]*в[- ]*н[- ]*ю[- ]*[кч]|д[- ]*е[- ]*р[- ]*ь[- ]*м[- ]*[аоуы]|д[- ]*е[- ]*р[- ]*[ьъ][- ]*м[- ]*[аоу]|д[- ]*[еи][- ]*р[- ]*ь[- ]*м[- ]*[а-яё-]*)|

# 7. Оскорбления женского рода (обновлено)
(?:с[- ]*[ую][- ]*[кч][- ]*[аинуекой]*|с[- ]*у[- ]*ч[- ]*[аиьо][- ]*[йея]|с[- ]*у[- ]*ч[- ]*к[- ]*[аиу]|с[- ]*у[- ]*ч[- ]*ь[- ]*[еяий]|с[- ]*т[- ]*е[- ]*р[- ]*в[- ]*[а-яё-]*|х[- ]*р[- ]*ы[- ]*ч[- ]*[а-яё-]*|
ш[- ]*м[- ]*а[- ]*р[- ]*[а-яё-]*|ш[- ]*в[- ]*а[- ]*л[- ]*[а-яё-]*|ш[- ]*л[- ]*ю[- ]*х[- ]*[а-яё-]*|б[- ]*л[- ]*я[- ]*д[- ]*[а-яё-]*|п[- ]*о[- ]*т[- ]*а[- ]*с[- ]*к[- ]*[а-яё-]*|п[- ]*р[- ]*[оа][- ]*л[- ]*[её][- ]*т[- ]*к[- ]*[а-яё-]*|с[- ]*у[- ]*ч[- ]*а[- ]*р[- ]*[а-яё-]*)|

# Еще
(?:д[- ]*а[- ]*в[- ]*а[- ]*л[- ]*к[- ]*[а-яё-]*|
   п[- ]*е[- ]*т[- ]*у[- ]*[хш][- ]*[а-яё-]*|
   р[- ]*у[- ]*к[- ]*о[- ]*б[- ]*л[- ]*у[- ]*д[- ]*[а-яё-]*|
   [а-яё-]*с[- ]*с[- ]*[аыу][- ]*[а-яё-]*|
   в[- ]*л[- ]*а[- ]*г[- ]*а[- ]*л[- ]*и[- ]*щ[- ]*[а-яё-]*|
   [а-яё-]*п[- ]*е[- ]*р[- ]*д[- ]*[а-яё-]*|
   [а-яё-]*п[- ]*[ё][- ]*р[- ]*н[- ]*[а-яё-]*|
   м[- ]*а[- ]*л[- ]*а[- ]*ф[- ]*ь[- ]*[а-яё-]*|
   г[- ]*о[- ]*м[- ]*и[- ]*к[- ]*[а-яё-]*|
   п[- ]*и[- ]*л[- ]*о[- ]*т[- ]*к[- ]*[а-яё-]*|
   а[- ]*н[- ]*у[- ]*с[- ]*[а-яё-]*|
   п[- ]*у[- ]*т[- ]*а[- ]*н[- ]*[ыа-]*[а-яё-]|
   ш[- ]*а[- ]*л[- ]*а[- ]*в[- ]*[а-яё-]*|
   м[- ]*о[- ]*ш[- ]*о[- ]*н[- ]*к[- ]*[а-яё-]*|
   е[- ]*л[- ]*д[- ]*[ао][- ]*[а-яё-]*|
   [а-яё-]*с[- ]*р[- ]*[еёиа][- ]*[тницклчвм][- ]*[а-яё-]*|
   [а-яё-]*с[- ]*р[- ]*[еёиау][- ]*|
   [а-яё-]*д[- ]*р[- ]*[еи][- ]*[сщ][- ]*[а-яё-]*|
   [а-яё-]*д[- ]*р[- ]*и[- ]*[сщ][- ]*[а-яё-]*|
   с[- ]*п[- ]*[еи][- ]*[р][- ]*[м][- ]*[а-яё-]*|
   д[- ]*[оа][- ]*[л][- ]*[б][- ]*[оа][- ]*[а-яё-]*|
   [ао][- ]*[н][- ]*[а][- ]*[л][- ]*[ауомыь][- ]*[а-яё-]*|
   [х][- ]*[р][- ]*[е][- ]*[н][- ]*[ьи][- ]*[а-яё-]*|
   [а-яё-]*с[- ]*[еёи][- ]*р[- ]*|
   [а-яё-]*с[- ]*[еёи][- ]*р[- ]*[с][- ]*[а-яё-]*|
   [а-яё-]*с[- ]*[еёи][- ]*р[- ]*[аиес][- ]*[тцлеиш][- ]*[а-яё-]*|
   [оа][- ]*[л][- ]*[ие][- ]*[г][- ]*[оа][- ]*[ф][- ]*[р][- ]*[а-яё-]*|
	[ш][- ]*[аои][- ]*[п][- ]*[ао][- ]*[в][- ]*[ь][- ]*[а-яё-]*|
    [ш][- ]*[п][- ]*[а][- ]*[к][- ]*[а-яё-]*|
    [х][- ]*[оа][- ]*[х][- ]*[о][- ]*[л][- ]*[а-яё-]*|
    [х][- ]*[оа][- ]*[х][- ]*[л][- ]*[а-яё-]*|
    ж[- ]*л[- ]*о[- ]*б[- ]*[а-яё-]*|
    т[- ]*в[- ]*[оа][- ]*р[- ]*[иь][- ]*[а-яё-]*|
	т[- ]*у[- ]*п[- ]*и[- ]*ц[- ]*[а-яё-]*|
    в[- ]*ы[- ]*р[- ]*[ао][- ]*д[- ]*о[- ]*к[- ]*[а-яё-]*|
    б[- ]*[оа][- ]*л[- ]*в[- ]*а[- ]*н[- ]*[а-яё-]*|
    о[- ]*л[- ]*у[- ]*х[- ]*[а-яё-]*|
    б[- ]*а[- ]*л[- ]*а[- ]*м[- ]*о[- ]*ш[- ]*к[- ]*[а-яё-]*|
    ч[- ]*у[- ]*р[- ]*к[- ]*[а-яё-]*|
    ж[- ]*и[- ]*д[- ]*[а-яё-]*|
    к[- ]*а[- ]*ц[- ]*а[- ]*п[- ]*[а-яё-]*|
    ч[- ]*у[- ]*х[- ]*а[- ]*н[- ]*[а-яё-]*|
    п[- ]*[еи][- ]*н[- ]*д[- ]*о[- ]*с[- ]*[а-яё-]*|
    б[- ]*а[- ]*н[- ]*д[- ]*е[- ]*р[- ]*[а-яё-]*|
    в[- ]*а[- ]*т[- ]*н[- ]*и[- ]*к[- ]*[а-яё-]*|
    б[- ]*и[- ]*о[- ]*м[- ]*а[- ]*с[- ]*с[- ]*[а-яё-]*|
    [а-яё-]*м[- ]*у[- ]*с[- ]*[оа][- ]*[р][- ]*[а-яё-]*|
    м[- ]*[еи][- ]*н[- ]*[т][- ]*[а-яё-]*|
    б[- ]*и[- ]*о[- ]*о[- ]*т[- ]*х[- ]*о[- ]*д[- ]*[а-яё-]*|
    [а-яё-]*м[- ]*о[- ]*ч[- ]*[аё][- ]*[а-яё-]*|
    к[- ]*а[- ]*л[- ]*[а-яё-]*|
    д[- ]*н[- ]*о[- ]*[а-яё-]*|
    о[- ]*т[- ]*р[- ]*е[- ]*б[- ]*ь[- ]*[ея][- ]*[а-яё-]*|
    т[- ]*р[- ]*о[- ]*л[- ]*[а-яё-]*|
    х[- ]*е[- ]*й[- ]*т[- ]*[еи][- ]*р[- ]*[а-яё-]*|
    к[- ]*л[- ]*о[- ]*у[- ]*н[- ]*[а-яё-]*|
    ш[- ]*[аи][- ]*р[- ]*л[- ]*[ао][- ]*т[- ]*а[- ]*н[- ]*[а-яё-]*|
    ш[- ]*[ие][- ]*з[- ]*[а-яё-]*|
    ю[- ]*р[- ]*о[- ]*д[- ]*и[- ]*в[- ]*[ыий][- ]*[а-яё-]*|
    у[- ]*[зс][- ]*к[- ]*[оа][- ]*г[- ]*л[- ]*а[- ]*з[- ]*[а-яё-]*|
    з[- ]*а[- ]*д[- ]*н[- ]*[ие][- ]*ц[- ]*[а-яё-]*|
    ч[- ]*у[- ]*ч[- ]*е[- ]*л[- ]*о[- ]*[а-яё-]*|
    г[- ]*а[- ]*д[- ]*[еи][- ]*н[- ]*[а-яё-]*|
    г[- ]*а[- ]*д[- ]*[а-яё-]*|
    б[- ]*е[- ]*з[- ]*м[- ]*о[- ]*з[- ]*г[- ]*л[- ]*[а-яё-]*|
    п[- ]*р[- ]*и[- ]*м[- ]*а[- ]*т[- ]*[а-яё-]*|
    о[- ]*б[- ]*е[- ]*з[- ]*ь[- ]*я[- ]*н[- ]*а-яё-]*|
    н[- ]*[еи][- ]*г[- ]*р[- ]*[а-яё-]*|
    н[- ]*[еи][- ]*г[- ]*г[- ]*е[- ]*р[- ]*[а-яё-]*|
    д[- ]*е[- ]*г[- ]*р[- ]*[ао][- ]*д[- ]*[а-яё-]*|
    д[- ]*е[- ]*г[- ]*р[- ]*о[- ]*[еи][- ]*д[- ]*[а-яё-]*|
    ж[- ]*и[- ]*р[- ]*т[- ]*р[- ]*е[- ]*с[- ]*[а-яё-]*|
    д[- ]*р[- ]*ы[- ]*[хщ][- ]*[а-яё-]*|
    н[- ]*и[- ]*щ[- ]*[еи][- ]*б[- ]*р[- ]*о[- ]*д[- ]*[а-яё-]*|
    н[- ]*и[- ]*щ[- ]*[еи][- ]*[ё][- ]*б[- ]*[а-яё-]*|
    м[- ]*а[- ]*к[- ]*а[- ]*к[- ]*[а-яё-]*|
    ж[- ]*и[- ]*р[- ]*[оа][- ]*б[- ]*а[- ]*с[- ]*[а-яё-]*|
    с[- ]*т[- ]*а[- ]*р[- ]*п[- ]*[её][- ]*р[- ]*[а-яё-]*|
    м[- ]*о[- ]*л[- ]*о[- ]*к[- ]*о[- ]*с[- ]*[а-яё-]*|
    ч[- ]*[ие][- ]*р[- ]*н[- ]*у[- ]*х[- ]*[а-яё-]*|
    [её][- ]*[п][- ]*|
    [её][- ]*[п][- ]*[т][- ]*[а-яё-]*|
    [й][- ]*[о][- ]*[п][- ]*[т][- ]*[а-яё-]*|
    [й][- ]*[о][- ]*[б][- ]*[а-яё-]*|
    [к][- ]*[у][- ]*[р][- ]*[в][- ]*[а-яё-]*|
    [к][- ]*[у][- ]*[р][- ]*[и][- ]*[ц][- ]*[а-яё-]*|
    б[- ]*а[- ]*б[- ]*[аыуомий-]*|
    ш[- ]*к[- ]*у[- ]*р[- ]*[аыуомий-]*|
    [а-яё-]*с[- ]*к[- ]*а[- ]*м[- ]*[илатьре-]*|
   	[иы][- ]*м[- ]*б[- ]*[еи][- ]*ц[- ]*[а-яё-]*|
    п[- ]*[ао][- ]*д[- ]*л[- ]*[оума]*|
    з[- ]*в[- ]*е[- ]*з[- ]*д[- ]*е[- ]*[лтжьиаш]*|
    ч[- ]*[её][- ]*р[- ]*т[- ]*|
    г[- ]*р[- ]*я[- ]*з[- ]*[а-яё-]*|
    г[- ]*е[- ]*[йию][- ]*[а-яё-]*|
    [а-яё-]*г[- ]*н[- ]*и[- ]*[еёлв][- ]*[а-яё-]*|
    [а-яё-]*с[- ]*к[- ]*у[- ]*ф[- ]*[а-яё-]*|
    м[- ]*е[- ]*ф[- ]*[а-яё-]*|
    о[- ]*т[- ]*с[- ]*т[- ]*о[- ]*й[- ]*|
    [оа][- ]*м[- ]*е[- ]*р[- ]*з[- ]*[а-яё-]*|
    [ам][- ]*м[- ]*е[- ]*р[- ]*[ыуома]|
    [а-яё-]*г[- ]*а[- ]*[дж][- ]*[а-яё-]*|
    о[- ]*б[- ]*и[- ]*ж[- ]*[еи][- ]*н[- ]*к[- ]*[а-яё-]*|
    ф[- ]*[ие][- ]*к[- ]*а[- ]*л[- ]*[а-яё-]*|
    [а-яё-]*р[- ]*ы[- ]*г[- ]*[а-яё-]*|
    [а-яё-]*б[- ]*л[- ]*[июе][- ]*[вю][- ]*[а-яё-]*|
    л[- ]*[ие][- ]*б[- ]*[ие][- ]*р[- ]*[ао][- ]*[сш][- ]*[а-яё-]*|
    п[- ]*[оа][- ]*р[- ]*а[- ]*ш[- ]*[а-яё-]*|
    п[- ]*[оа][- ]*д[- ]*л[- ]*[ю][- ]*[а-яё-]*|
    [а-яё-]*х[- ]*у[- ]*л[- ]*[еи][- ]*[а-яё-]*|
    [а-яё-]*х[- ]*у[- ]*[еюйё][- ]*[- ]*[а-яё-]*|
    б[- ]*л[- ]*[я][- ]*[- ]*[а-яё-]*|
    [а-яё-]*т[- ]*р[- ]*а[- ]*х[- ]*[- ]*[а-яё-]*|
    [а-яё-]*[оа][- ]*н[- ]*[оа][- ]*[н][- ]*[а-яё-]*|
    [а-яё-]*ц[- ]*е[- ]*л[- ]*к[- ]*[а-яё-]*|
    [а-яё-]*д[- ]*е[- ]*в[- ]*с[- ]*т[- ]*в[- ]*[а-яё-]*|
    с[- ]*е[- ]*к[- ]*с[- ]*[а-яё-]*|
    л[- ]*г[- ]*б[- ]*т[- ]*|
    в[- ]*и[- ]*ч[- ]*[а-яё-]*|
    с[- ]*п[- ]*[ие][- ]*д[- ]*[а-яё-]*|
    с[- ]*[ие][- ]*ф[- ]*[аие][- ]*[кл][- ]*[а-яё-]*|
    щ[- ]*[ие][- ]*г[- ]*о[- ]*л[- ]*[а-яё-]*|
    щ[- ]*[ие][- ]*г[- ]*л[- ]*[а-яё-]*|
    д[- ]*я[- ]*т[- ]*е[- ]*л[- ]*|
    д[- ]*я[- ]*т[- ]*л[- ]*[а-яё-]*|
    б[- ]*а[- ]*л[- ]*б[- ]*е[- ]*с[- ]*[а-яё-]*|
    б[- ]*а[- ]*р[- ]*ы[- ]*[гж][- ]*[а-яё-]*|
    б[- ]*р[- ]*е[- ]*х[- ]*у[- ]*н[- ]*[а-яё-]*|
    б[- ]*р[- ]*е[- ]*х[- ]*н[- ]*[а-яё-]*|
    г[- ]*р[- ]*у[- ]*б[- ]*[а-яё-]*|
    з[- ]*ю[- ]*з[- ]*[а-яё-]*|
    к[- ]*р[- ]*и[- ]*в[- ]*л[- ]*я[- ]*[а-яё-]*|
    п[- ]*о[- ]*з[- ]*[её][- ]*р[- ]*|
    к[- ]*в[- ]*а[- ]*д[- ]*р[- ]*о[- ]*б[- ]*е[- ]*р[- ]*[а-яё-]*|
    м[- ]*е[- ]*г[- ]*е[- ]*р[- ]*[а-яё-]*|
    н[- ]*а[- ]*х[- ]*а[- ]*л[а-яё-]*|
    [ао][- ]*б[- ]*[ао][- ]*л[- ]*д[- ]*у[- ]*[а-яё-]*|
    [а-яё-]*п[- ]*а[- ]*к[- ]*о[- ]*с[- ]*т[- ]*[а-яё-]*|
    п[- ]*а[- ]*с[- ]*к[- ]*у[- ]*д[- ]*[а-яё-]*|
    п[- ]*и[- ]*ж[- ]*о[- ]*н[- ]*[а-яё-]*|
    п[- ]*[ао][- ]*г[- ]*[а][- ]*н[- ]*[а-яё-]*|
    
    п[- ]*[ао][- ]*т[- ]*[ао][- ]*с[- ]*к[- ]*у[- ]*х[- ]*[а-яё-]*| 
	п[- ]*р[- ]*[ао][- ]*ф[- ]*у[- ]*р[- ]*[а-яё-]*|          
	п[- ]*р[- ]*[ао][- ]*ф[- ]*у[- ]*р[- ]*с[- ]*е[- ]*т[- ]*к[- ]*[а-яё-]*|  
	п[- ]*р[- ]*[fj][- ]*ш[- ]*м[- ]*а[- ]*н[- ]*д[- ]*о[- ]*в[- ]*к[- ]*[а-яё-]*|  
	ш[- ]*[оа][- ]*л[- ]*а[- ]*в[- ]*[а-яё-]*|                  
	ш[- ]*л[- ]*ю[- ]*[шх][- ]*[а-яё-]*|                        
	ш[- ]*м[- ]*а[- ]*р[- ]*[а-яё-]*|                       
	р[- ]*а[- ]*с[- ]*ш[- ]*и[- ]*в[- ]*о[- ]*х[- ]*[а-яё-]*|  
    ш[- ]*[оа][- ]*б[- ]*о[- ]*л[- ]*д[- ]*[а-яё-]*| 

    р[- ]*в[- ]*а[- ]*н[- ]*[ьь][- ]*[а-яё-]*|              
	с[- ]*т[- ]*е[- ]*р[- ]*в[- ]*е[- ]*ц[- ]*[а-яё-]*|      
	с[- ]*т[- ]*е[- ]*р[- ]*в[- ]*[а-яё-]*|                 
	с[- ]*в[- ]*о[- ]*л[- ]*о[- ]*ч[- ]*[уньь][- ]*[а-яё-]*|   
	[а-яё-]*х[- ]*а[- ]*м[- ]*[а-яё-]*|                              
	х[- ]*м[- ]*ы[- ]*р[- ]*[ьь][- ]*[а-яё-]*|              
	х[- ]*[ао][- ]*л[- ]*у[- ]*й[- ]*[а-яё-]*|                  
	ш[- ]*[оаеиы][- ]*н[- ]*т[- ]*р[- ]*[ао][- ]*п[- ]*[а-яё-]*|     
	ш[- ]*е[- ]*л[- ]*ь[- ]*м[- ]*[а-яё-]*|                  
	ш[- ]*п[- ]*[оа][- ]*н[- ]*[а-яё-]*|                       
	у[- ]*р[- ]*к[- ]*[а-яё-]*|                              
	ф[- ]*р[- ]*а[- ]*е[- ]*р[- ]*[а-яё-]*|                 
	ф[- ]*у[- ]*ф[- ]*л[- ]*[аыо][- ]*[а-яё-]*|  
    ф[- ]*у[- ]*ф[- ]*е[- ]*л[- ]*[а-яё-]*| 
    
    ч[- ]*у[- ]*х[- ]*а[- ]*н[- ]*[а-яё-]*| 
    ч[- ]*у[- ]*ш[- ]*п[- ]*а[- ]*н[- ]*[а-яё-]*| 
    
    р[- ]*а[- ]*з[- ]*з[- ]*я[- ]*в[- ]*[а-яё-]*|          
	т[- ]*р[- ]*е[- ]*п[- ]*л[- ]*[а-яё-]*|           
	ф[- ]*и[- ]*н[- ]*т[- ]*и[- ]*ф[- ]*л[- ]*ю[- ]*ш[- ]*к[- ]*[а-яё-]*|  
	ч[- ]*у[- ]*в[- ]*ы[- ]*р[- ]*л[- ]*[а-яё-]*|      
	ч[- ]*у[- ]*ч[- ]*е[- ]*л[- ]*[а-яё-]*|           
	ш[- ]*в[- ]*а[- ]*б[- ]*р[- ]*[а-яё-]*|                
	ш[- ]*е[- ]*л[- ]*у[- ]*п[- ]*о[- ]*н[- ]*[ьь][- ]*[а-яё-]*|  
	ш[- ]*и[- ]*б[- ]*з[- ]*д[- ]*и[- ]*к[- ]*[а-яё-]*|      
	ш[- ]*к[- ]*у[- ]*р[- ]*[а-яё-]*|                       
	ш[- ]*у[- ]*ш[- ]*е[- ]*р[- ]*[а-яё-]*| 
    ш[- ]*м[- ]*[ао][- ]*к[- ]*[оа][- ]*д[- ]*[а-яё-]*| 

	г[- ]*[еи][- ]*м[- ]*[ао][- ]*р[- ]*р[- ]*о[- ]*й[- ]*[а-яё-]*| 
	д[- ]*[еи][- ]*с[- ]*т[- ]*р[- ]*о[- ]*ф[- ]*[а-яё-]*|  
	я[- ]*з[- ]*в[- ]*[а-яё-]*|                             
    
    к[- ]*в[- ]*а[- ]*с[- ]*и[- ]*[а-яё-]*|  
	п[- ]*р[- ]*о[- ]*п[- ]*о[- ]*й[- ]*ц[- ]*[а-яё-]*|      
	х[- ]*[оа][- ]*н[- ]*ы[- ]*г[- ]*[а-яё-]*|    
    
    а[- ]*л[- ]*к[- ]*о[- ]*н[- ]*а[- ]*[вф][- ]*т[- ]*[а-яё-]*|  
	б[- ]*у[- ]*з[- ]*и[- ]*т[- ]*[ьь][- ]*[а-яё-]*|  
    
    л[- ]*е[- ]*г[- ]*а[- ]*в[- ]*[а-яё-]*| 
    с[- ]*о[- ]*л[- ]*д[- ]*а[- ]*ф[- ]*о[- ]*н[- ]*[а-яё-]*|
    с[- ]*у[- ]*т[- ]*[ие][- ]*н[- ]*[её][- ]*р[- ]*[а-яё-]*|
    в[- ]*ш[- ]*и[- ]*в[- ]*[а-яё-]*|
    о[- ]*л[- ]*е[- ]*н[- ]*[а-яё-]*|
    к[- ]*р[- ]*ы[- ]*с[- ]*[а-яё-]*|
    м[- ]*ы[- ]*ш[- ]*[а-яё-]*|
    ш[- ]*[оа][- ]*ф[- ]*е[- ]*|
    [а-яё-]*п[- ]*[оа][- ]*д[- ]*л[- ]*[а-яё-]*|
    [а-яё-]*с[- ]*т[- ]*[её][- ]*б[- ]*[а-яё-]*|
    п[- ]*[оа][- ]*д[- ]*с[- ]*т[- ]*и[- ]*л[- ]*к[а-яё-]*|
    н[- ]*е[- ]*ж[- ]*[еи][- ]*н[- ]*[а-яё-]*|
    б[- ]*р[- ]*е[- ]*д[- ]*[а-яё-]*|
    п[- ]*у[- ]*к[- ]*[а-яё-]*|
    [а-яё-]*к[- ]*[еа][- ]*к[- ]*[аи][- ]*[тшхеюа-яё-]*|
    а[- ]*с[- ]*п[- ]*и[- ]*д[- ]*[а-яё-]*|
    ц[- ]*е[- ]*л[- ]*о[- ]*ч[- ]*к[а-яё-]*|
    с[- ]*[ао][- ]*ч[- ]*к[- ]*[оуаи][- ]*[а-яё-]*|
    [ао][- ]*ч[- ]*к[- ]*[оау][- ]*[а-яё-]*|
    с[- ]*[ао][- ]*ч[- ]*[оеё][- ]*[а-яё-]*|
    н[- ]*[оа][- ]*в[- ]*о[- ]*з[- ]*[а-яё-]*|
    п[- ]*с[- ]*[еи][- ]*х[- ]*[дбл][- ]*[а-яё-]*|
    п[- ]*с[- ]*и[- ]*х[- ]*[ауий]*|
    п[- ]*с[- ]*и[- ]*х[- ]*и[- ]*ч[- ]*[а-яё-]*|
    ч[- ]*е[- ]*л[- ]*[ыа]|
    ч[- ]*е[- ]*л[- ]*|
    п[- ]*о[- ]*п[- ]*[ыа]|
    х[- ]*р[- ]*о[- ]*м[- ]*о[- ]*с[- ]*о[- ]*м[- ]*[а-яё-]*|
    п[- ]*[еи][- ]*т[- ]*у[- ]*х[- ]*[а-яё-]*|
    н[- ]*и[- ]*г[- ]*[ие][- ]*р[- ]*[а-яё-]*|
    н[- ]*и[- ]*г[- ]*а[- ]*|
    м[- ]*а[- ]*т[- ]*ь[- ]*[а-яё-]*|
    м[- ]*а[- ]*т[- ]*е[- ]*р[- ]*[ья][- ]*[а-яё-]*|
    м[- ]*а[- ]*м[- ]*к[- ]*[а-яё-]*|
    [а-яё-]*с[- ]*[еёи][- ]*р[- ]*[а-яё-]*|
    н[- ]*а[- ]*г[- ]*н[- ]*у[- ]*[а-яё-]*|
    [а-яё-]*п[- ]*и[- ]*с[- ]*[ьаеюё][- ]*[еклнчюш][- ]*[а-яё-]*|
    [а-яё-]*к[- ]*у[- ]*р[- ]*[яи][- ]*т[- ]*[а-яё-]*|
    [а-яё-]*к[- ]*у[- ]*р[- ]*[еи][- ]*н[- ]*[а-яё-]*|
    [а-яё-]*к[- ]*у[- ]*р[- ]*[иею][- ]*[а-яё-]*|
    к[- ]*[ао][- ]*с[- ]*я[- ]*к[- ]*[а-яё-]*|
    с[- ]*[еыи][- ]*г[- ]*[оа][- ]*р[- ]*[а-яё-]*|
    ч[- ]*[ие][- ]*л[- ]*ю[- ]*х[- ]*[а-яё-]*|
    н[- ]*е[- ]*р[- ]*у[- ]*с[- ]*ь[- ]*[а-яё-]*|
    с[- ]*у[- ]*[эи][- ]*т[- ]*[а-яё-]*|
    к[- ]*а[- ]*л[- ]*ь[- ]*я[- ]*н[- ]*[а-яё-]*|
    к[- ]*а[- ]*л[- ]*и[- ]*к[- ]*[а-яё-]*|
    п[- ]*и[- ]*в[- ]*[кчаоы][- ]*[а-яё-]*|
    в[- ]*о[- ]*д[- ]*к[- ]*[а-яё-]*|
    в[- ]*о[- ]*д[- ]*я[- ]*р[- ]*[а-яё-]*|
    с[- ]*[оа][- ]*м[- ]*[ао][- ]*г[- ]*о[- ]*н[- ]*[а-яё-]*|
    б[- ]*у[- ]*р[- ]*д[- ]*а[- ]*[а-яё-]*|
    б[- ]*а[- ]*л[- ]*а[- ]*н[- ]*д[- ]*[а-яё-]*|
    [а-яё-]*к[- ]*а[- ]*з[- ]*и[- ]*[нк][- ]*[а-яё-]*|
    [а-яё-]*с[- ]*т[- ]*а[- ]*в[- ]*к[- ]*[а-яё-]*|
    [а-яё-]*с[- ]*т[- ]*а[- ]*в[- ]*о[- ]*[чк]*[а-яё-]*|
    [а-яё-]*б[- ]*[эеё][- ]*т[- ]*[а-яё-]*|
    [а-яё-]*ь[- ]*[эеё][- ]*т[- ]*[а-яё-]*|
    [а-яё-]*в[- ]*[эеё][- ]*т[- ]*[а-яё-]*|
    [а-яё-]*п[- ]*о[- ]*к[- ]*е[- ]*р[- ]*[а-яё-]*|
    [а-яё-]*р[- ]*о[- ]*к[- ]*е[- ]*[яг][- ]*[а-яё-]*|
    [а-яё-]*б[- ]*у[- ]*к[- ]*м[- ]*е[- ]*к[- ]*[а-яё-]*|
    [а-яё-]*с[- ]*а[- ]*с[- ]*и[- ]*н[- ]*о[- ]*[а-яё-]*|
    [а-яё-]*б[- ]*л[- ]*ю[- ]*[а-яё-]*|
    [а-яё-]*х[- ]*р[- ]*ю[- ]*[а-яё-]*|
    [а-яё-]*д[- ]*о[- ]*д[- ]*[а-яё-]*|
    [а-яё-]*к[- ]*р[- ]*и[- ]*н[- ]*ж[- ]*[а-яё-]*|
    а[- ]*у[- ]*[еф][- ]*[а-яё-]*|
    ж[- ]*и[- ]*р[- ]*н[- ]*[а-яё-]*|
    ж[- ]*и[- ]*р[- ]*[а-яё-]*|
    [а-яё-]*ж[- ]*и[- ]*р[- ]*[а-яё-]*|
    л[- ]*[ие][- ]*к[- ]*[еёи][- ]*р[- ]*[а-яё-]*|
    к[- ]*о[- ]*н[- ]*ь[- ]*я[- ]*[кч][- ]*[а-яё-]*|
    в[- ]*и[- ]*с[- ]*к[- ]*и[- ]*|
    в[- ]*и[- ]*с[- ]*к[- ]*а[- ]*р[- ]*[а-яё-]*|
    с[- ]*и[- ]*к[- ]*а[- ]*[юяте][- ]*[а-яё-]*|
    с[- ]*и[- ]*с[- ]*[ьия][- ]*[а-яё-]*|
    а[- ]*н[- ]*а[- ]*л[- ]*[ыоьау][- ]*[а-яё-]*|
    л[- ]*[еи][- ]*с[- ]*б[- ]*[а-яё-]*|
    б[- ]*д[- ]*с[- ]*м[- ]*[а-яё-]*|
    ц[- ]*е[- ]*л[- ]*[ао][- ]*ч[- ]*к[- ]*|
    к[- ]*у[- ]*н[- ]*[еия][- ]*[а-яё-]*|
    б[- ]*р[- ][её][- ]*в[- ]*н*[а-яё-]*|
    о[- ]*п[- ]*и[- ]*а[- ]*т[- ]*[а-яё-]*|
    м[- ]*[ао][- ]*р[- ]*ф[- ]*и[- ]*н[- ]*[а-яё-]*|
    к[- ]*о[- ]*д[- ]*е[- ]*и[- ]*н[- ]*[а-яё-]*|
    д[- ]*и[- ]*г[- ]*и[- ]*д[- ]*р[- ]*о[- ]*к[- ]*о[- ]*д[- ]*е[- ]*и[- ]*н[- ]*[а-яё-]*|
    [а-яё-]*а[- ]*м[- ]*ф[- ]*[ие][- ]*т[- ]*[оа][- ]*м[- ]*и[- ]*н[- ]*[а-яё-]*|
    [а-яё-]*э[- ]*ф[- ]*[ие][- ]*д[- ]*р[- ]*и[- ]*н*[- ][а-яё-]*|
    г[- ]*а[- ]*ш[- ]*[а-яё-]*|
    м[- ]*е[- ]*т[- ]*[ао][- ]*д[- ]*о[- ]*н[- ]*[а-яё-]*|
    л[- ]*с[- ]*д[- ]*[а-яё-]*|
    к[- ]*а[- ]*н[- ]*а[- ]*б[- ]*и[- ]*с[- ]*[а-яё-]*|
    к[- ]*л[- ]*[ао][- ]*ф[- ]*е[- ]*л[- ]*и[- ]*н[- ]*[а-яё-]*|
    п[- ]*[оа][- ]*р[- ]*[оа][- ]*ц[- ]*[еи][- ]*т[- ]*[оа][- ]*м[- ]*о[- ]*л[- ]*[а-яё-]*|
    н[- ]*[еи][- ]*к[- ]*[ао][- ]*т[- ]*и[- ]*н[- ]*[а-яё-]*|
    а[- ]*л[- ]*к[- ]*[ао][- ]*г[- ]*о[- ]*л[- ]*[а-яё-]*|
    с[- ]*н[- ]*ю[- ]*с[- ]*[а-яё-]*|
    к[- ]*а[- ]*н[- ]*а[- ]*б[- ]*[а-яё-]*|
    о[- ]*п[- ]*и[- ]*[уаой][- ]*[а-яё-]*|
    к[- ]*о[- ]*н[- ]*о[- ]*п[- ]*л[- ]*[а-яё-]*|
    п[- ]*л[- ]*е[- ]*с[- ]*е[- ]*н[- ]*[а-яё-]*|
    м[- ]*а[- ]*с[- ]*т[- ]*у[- ]*р[- ]*б[- ]*[а-яё-]*|
    ч[- ]*с[- ]*в[- ]*[а-яё-]*|
    [а-яё-]*л[- ]*и[- ]*[зж][- ]*[аеиу][- ]*[тн][- ]*[а-яё-]*|
    т[- ]*я[- ]*н[- ]*|
    т[- ]*я[- ]*н[- ]*к[- ]*[а-яё-]*|
    т[- ]*я[- ]*н[- ]*о[- ]*ч[- ]*к[- ]*[а-яё-]*|
    б[- ]*л[- ]*е[- ]*д[- ]*[а-яё-]*|
    т[- ]*[оа][- ]*р[- ]*е[- ]*л[- ]*[ао][- ]*ч[- ]*н[- ]*[ие][- ]*ц[- ]*[а-яё-]*|
    п[- ]*а[- ]*д[- ]*а[- ]*л[- ]*ь[- ]*[а-яё-]*|
    т[- ]*р[- ]*а[- ]*н[- ]*с[- ]*в[- ]*и[- ]*с[- ]*т[- ]*[а-яё-]*|
    т[- ]*р[- ]*а[- ]*н[- ]*с[- ]*г[- ]*е[- ]*н[- ]*д[- ]*[а-яё-]*|
    п[- ]*о[- ]*п[- ]*к[- ]*[а-яё-]*|
    н[- ]*[уо][- ]*л[- ]*[иеё][- ]*в[- ]*[а-яё-]*|
    с[- ]*к[- ]*о[- ]*р[- ]*о[- ]*с[- ]*т[- ]*р[- ]*е[- ]*л[- ]*[а-яё-]*|
    э[- ]*р[- ]*е[- ]*к[- ]*ц[- ]*и[- ]*[а-яё-]*|
    э[- ]*р[- ]*о[- ]*г[- ]*е[- ]*н[- ]*[а-яё-]*|
    э[- ]*р[- ]*о[- ]*т[- ]*[а-яё-]*|
    с[- ]*т[- ]*р[- ]*и[- ]*п[- ]*т[- ]*и[- ]*[а-яё-]*|
    с[- ]*т[- ]*р[- ]*и[- ]*п[- ]*у[- ]*[а-яё-]*|
    в[- ]*а[- ]*ф[- ]*л[- ]*[иея][- ]*[а-яё-]*|
    в[- ]*а[- ]*ф[- ]*е[- ]*л[- ]*[а-яё-]*|
    к[- ]*р[- ]*и[- ]*н[- ]*ж[- ]*[а-яё-]*|
    в[- ]*а[- ]*л[- ]*е[- ]*н[- ]*о[- ]*к[- ]*[а-яё-]*|
    т[- ]*а[- ]*л[- ]*и[- ]*б[- ]*[а-яё-]*|
    т[- ]*е[- ]*р[- ]*р[- ]*о[- ]*р[- ]*[а-яё-]*|
    и[- ]*г[- ]*и[- ]*л[- ]*[а-яё-]*|
    к[- ]*л[- ]*и[- ]*т[- ]*о[- ]*р[- ]*[а-яё-]*|
    к[- ]*л[- ]*и[- ]*т[- ]*р[- ]*[а-яё-]*|
    в[- ]*и[- ]*а[- ]*г[- ]*р[- ]*[а-яё-]*|
    э[- ]*я[- ]*к[- ]*у[- ]*л[- ]*я[- ]*ц[- ]*[а-яё-]*|
    а[- ]*н[- ]*у[- ]*с[- ]*[а-яё-]*|
    л[- ]*[еи][- ]*б[- ]*и[- ]*д[- ]*[а-яё-]*|
    [а-яё-]*м[- ]*[еи][- ]*н[- ]*с[- ]*т[- ]*р[- ]*[а-яё-]*|
    п[- ]*м[- ]*с[- ]*[а-яё-]*|
    з[- ]*а[- ]*п[- ]*о[- ]*р[- ]*[а-яё-]*|
    и[- ]*н[- ]*ц[- ]*е[- ]*л[- ]*[а-яё-]*|
    з[- ]*а[- ]*п[- ]*а[- ]*[хш][- ]*[а-яё-]*|
    п[- ]*е[- ]*р[- ]*е[- ]*п[- ]*и[- ]*х[- ]*[а-яё-]*|
    с[- ]*т[- ]*о[- ]*я[- ]*к[- ]*[а-яё-]*|
    [а-яё-]*х[- ]*р[- ]*е[- ]*н[- ]*[а-яё-]*|
    м[- ]*[ао][- ]*к[- ]*р[- ]*о[- ]*щ[- ]*[её][- ]*л[- ]*к[- ]*[а-яё-]*|
    в[- ]*е[- ]*б[- ]*к[- ]*а[- ]*м[- ]*[а-яё-]*|
    э[- ]*с[- ]*к[- ]*о[- ]*р[- ]*т[- ]*[а-яё-]*|
    б[- ]*и[- ]*с[- ]*е[- ]*к[- ]*с[- ]*[а-яё-]*|
    т[- ]*[её][- ]*л[- ]*к[- ]*[а-яё-]*|
    б[- ]*ы[- ]*д[- ]*л[- ]*[а-яё-]*|
    х[- ]*[ие][- ]*т[- ]*р[- ]*[ао][- ]*[жп][- ]*о[- ]*п[- ]*[а-яё-]*|
    в[- ]*р[- ]*о[- ]*т[- ]*[а-яё-]*|
    р[- ]*о[- ]*т[- ]*[эа][- ]*н[- ]*[а-яё-]*|
    р[- ]*о[- ]*т[- ]*[еи][- ]*к[- ]*[а-яё-]*|
    р[- ]*о[- ]*т[- ]*|
    ц[- ]*и[- ]*р[- ]*к[- ]*[а-яё-]*|
    п[- ]*[оа][- ]*р[- ]*[оа][- ]*н[- ]*[оа][- ]*[а-яё-]*|
    с[- ]*е[- ]*к[- ]*т[- ]*а[- ]*[а-яё-]*|
    с[- ]*т[- ]*а[- ]*д[- ]*[оауми-]*|
    м[- ]*[оу][- ]*х[- ]*[ао][- ]*м[- ]*о[- ]*р[- ]*[а-яё-]*|
    [ао][- ]*х[- ]*[еи][- ]*н[- ]*е[- ]*[а-яё-]*|
    д[- ]*ы[- ]*р[- ]*[а-яё-]*|
    п[- ]*[ао][- ]*н[- ]*т[- ]*[ао][- ]*р[- ]*е[- ]*з[- ]*[а-яё-]*|
    [а-яё-]*н[- ]*ы[- ]*т[- ]*[а-яё-]*|
    # [а-яё-]*н[- ]*о[- ]*[йеюя][- ]*[а-яё-]*|
    с[- ]*т[- ]*у[- ]*к[- ]*а[- ]*ч[- ]*[а-яё-]*|
    б[- ]*е[- ]*з[- ]*д[- ]*а[- ]*р[- ]*[а-яё-]*|
    щ[- ]*е[- ]*г[- ]*о[- ]*л[- ]*|
    щ[- ]*е[- ]*г[- ]*л[- ]*[а-яё-]*|
    [оа][- ]*п[- ]*л[- ]*[оа][- ]*д[- ]*[ао][- ]*т[- ]*в[- ]*[ао][- ]*р[- ]*[а-яё-]*|
    к[- ]*у[- ]*к[- ]*у[- ]*[а-яё-]*|
    л[- ]*у[- ]*к[- ]*а[- ]*в[- ]*[а-яё-]*|
    ш[- ]*[оа][- ]*м[- ]*п[- ]*а[- ]*н[- ]*с[- ]*[а-яё-]*|
    ч[- ]*[еи][- ]*н[- ]*[оу][- ]*ш[- ]*[а-яё-]*|
    ч[- ]*[иеё][- ]*р[- ]*к[- ]*[оа][- ]*ш[- ]*[а-яё-]*|
    б[- ]*у[- ]*р[- ]*ж[- ]*у[- ]*[ийюя][- ]*[а-яё-]*|
    л[- ]*[еи][- ]*з[- ]*[ао][- ]*б[- ]*л[- ]*[ею][- ]*д[- ]*[а-яё-]*|
    к[- ]*у[- ]*р[- ]*т[- ]*[еи][- ]*з[- ]*а[- ]*н[- ]*[а-яё-]*|
    п[- ]*[оа][- ]*р[- ]*[оа][- ]*с[- ]*[еёя][- ]*[а-яё-]*|
    х[- ]*р[- ]*ю[- ]*[а-яё-]*|
    р[- ]*у[- ]*х[- ]*л[- ]*я[- ]*д[- ]*[а-яё-]*|
    к[- ]*л[- ]*я[- ]*ч[- ]*[а-яё-]*|
    п[- ]*у[- ]*ч[- ]*и[- ]*т[- ]*[а-яё-]*|
    п[- ]*р[- ]*о[- ]*к[- ]*а[- ]*з[- ]*н[- ]*и[- ]*[а-яё-]*|
    б[- ]*[оа][- ]*л[- ]*[оа][- ]*б[- ]*о[- ]*л[- ]*[а-яё-]*|
    з[- ]*[эе][- ]*к[- ]*[а-яё-]*|
    р[- ]*а[- ]*к[- ]*[а-яё-]*|
    с[- ]*о[- ]*с[- ]*о[- ]*к[- ]*[а-яё-]*|
    с[- ]*о[- ]*с[- ]*к[- ]*[а-яё-]*|
    г[- ]*р[- ]*у[- ]*д[- ]*[а-яё-]*|
    и[- ]*н[- ]*д[- ]*и[- ]*к[- ]*[а-яё-]*|
    с[- ]*а[- ]*т[- ]*и[- ]*в[- ]*[а-яё-]*|
    я[- ]*[ий][- ]*ц[- ]*[а-яё-]*|
    й[- ]*а[- ]*[ий][- ]*ц[- ]*[а-яё-]*|
    м[- ]*[оа][- ]*л[- ]*[оа][- ]*ч[- ]*к[- ]*[а-яё-]*|
    п[- ]*[ао][- ]*н[- ]*о[- ]*с[- ]*[а-яё-]*|
    п[- ]*з[- ]*д[- ]*ц[- ]*[а-яё-]*|
    в[- ]*ы[- ]*к[- ]*[еи][- ]*д[- ]*ы[- ]*ш[- ]*[а-яё-]*|
    м[- ]*[оа][- ]*р[- ]*[оа][- ]*з[- ]*м[- ]*[а-яё-]*|
    ч[- ]*у[- ]*р[- ]*о[- ]*к[- ]*[а-яё-]*|
    т[- ]*[оа][- ]*б[- ]*л[- ]*е[- ]*т[- ]*[а-яё-]*|
    л[- ]*а[- ]*х[- ]*[а-яё-]*|
    ш[- ]*а[- ]*ш[- ]*а[- ]*[а-яё-]*|
    н[- ]*ю[- ]*х[- ]*а[- ]*ч[- ]*[а-яё-]*|
    ш[- ]*о[- ]*б[- ]*л[- ]*[а-яё-]*|
    [а-яё-]*в[- ]*[ао][- ]*д[- ]*и[- ]*л[- ]*[а-яё-]*|
    к[- ]*р[- ]*[еи][- ]*в[- ]*[оа][- ]*н[- ]*о[- ]*г[- ]*[а-яё-]*|
    к[- ]*р[- ]*[еи][- ]*в[- ]*[оа][- ]*р[- ]*у[- ]*[чк][- ]*[а-яё-]*|
    к[- ]*р[- ]*[еи][- ]*в[- ]*[оа][- ]*з[- ]*а[- ]*д[- ]*[а-яё-]*|
    в[- ]*[эй][- ]*й[- ]*п[- ]*[а-яё-]*|
    э[- ]*л[- ]*е[- ]*к[- ]*т[- ]*р[- ]*о[- ]*н[- ]*к[- ]*[а-яё-]*|
    т[- ]*а[- ]*б[- ]*а[- ]*[чк][- ]*[а-яё-]*|
    к[- ]*[ао][- ]*ч[- ]*е[- ]*р[- ]*ы[- ]*ж[- ]*к[- ]*[а-яё-]*|
    п[- ]*[оа][- ]*п[- ]*р[- ]*[оа][- ]*ш[- ]*а[- ]*й[- ]*[а-яё-]*|
    в[- ]*[оа][- ]*л[- ]*[оа][- ]*с[- ]*н[- ]*[а-яё-]*|
    г[- ]*[оа][- ]*н[- ]*д[- ]*у[- ]*р[- ]*а[- ]*с[- ]*[а-яё-]*|
    у[- ]*к[- ]*р[- ]*о[- ]*п[- ]*[а-яё-]*|
    з[- ]*[ие][- ]*л[- ]*[еи][- ]*б[- ]*о[- ]*б[- ]*[а-яё-]*|
    б[- ]*[еи][- ]*д[- ]*о[- ]*н[- ]*[а-яё-]*|
    с[- ]*в[- ]*[её][- ]*р[- ]*т[- ]*о[- ]*к[- ]*[а-яё-]*|
    д[- ]*[еи][- ]*р[- ]*[ие][- ]*в[- ]*е[- ]*н[- ]*щ[- ]*[а-яё-]*|
    с[- ]*в[- ]*и[- ]*н[- ]*о[- ]*т[- ]*[а-яё-]*|
    к[- ]*и[- ]*д[- ]*а[- ]*л[- ]*[оа][- ]*[а-яё-]*|
    к[- ]*р[- ]*е[- ]*м[- ]*п[- ]*а[- ]*[а-яё-]*|
    в[- ]*о[- ]*д[- ]*н[- ]*и[- ]*к[- ]*[а-яё-]*|
    а[- ]*[фв][- ]*г[- ]*а[- ]*н[- ]*к[- ]*[а-яё-]*|
    ш[- ]*и[- ]*ш[- ]*а[- ]*р[- ]*[а-яё-]*|
    ш[- ]*и[- ]*ш[- ]*л[- ]*[а-яё-]*|
    и[- ]*с[- ]*л[- ]*а[- ]*м[- ]*и[- ]*с[- ]*т[- ]*[а-яё-]*|
    и[- ]*с[- ]*л[- ]*а[- ]*м[- ]*с[- ]*к[- ]*[а-яё-]*|
    т[- ]*в[- ]*[её][- ]*р[- ]*д[- ]*о[- ]*л[- ]*о[- ]*б[- ]*[а-яё-]*|
    к[- ]*о[- ]*с[- ]*т[- ]*о[- ]*л[- ]*о[- ]*м[- ]*[а-яё-]*|
    р[- ]*а[- ]*д[- ]*и[- ]*к[- ]*а[- ]*л[- ]*[а-яё-]*|
    [оа][- ]*г[- ]*у[- ]*з[- ]*[а-яё-]*|
    б[- ]*[оа][- ]*р[- ]*[оа][- ]*д[- ]*а[- ]*ч[- ]*[а-яё-]*|
    [оа][- ]*г[- ]*р[- ]*ы[- ]*з[- ]*[а-яё-]*|
    и[- ]*н[- ]*в[- ]*[оа][- ]*л[- ]*и[- ]*д[- ]*[а-яё-]*|
    к[- ]*[оа][- ]*л[- ]*е[- ]*к[- ]*[а-яё-]*|
    к[- ]*р[- ]*о[- ]*в[- ]*о[- ]*п[- ]*и[- ]*й[- ]*ц[- ]*[а-яё-]*|
    б[- ]*ы[- ]*ч[- ]*[а-яё-]*|
    б[- ]*ы[- ]*к[- ]*[оа][- ]*в[- ]*а[- ]*т[- ]*[а-яё-]*|
    [а-яё-]*к[- ]*у[- ]*й[- ]*[а-яё-]*|
    г[- ]*р[- ]*у[- ]*п[- ]*п[- ]*о[- ]*в[- ]*у[- ]*х[- ]*[а-яё-]*|
    к[- ]*л[- ]*а[- ]*д[- ]*м[- ]*[еэ][- ]*н[- ]*[а-яё-]*|
    з[- ]*а[- ]*к[- ]*л[- ]*а[- ]*д[- ]*ч[- ]*и[- ]*к[- ]*[а-яё-]*|
    м[- ]*е[- ]*р[- ]*т[- ]*в[- ]*е[- ]*[а-яё-]*|
    п[- ]*о[- ]*й[- ]*л[- ]*[а-яё-]*|
    х[- ]*у[- ]*н[- ]*т[- ]*[а-яё-]*|
    н[- ]*а[- ]*с[- ]*в[- ]*а[- ]*[а-яё-]*|
    н[- ]*а[- ]*с[- ]*и[- ]*к[- ]*[а-яё-]*|
    б[- ]*л[- ]*[оа][- ]*[хш][- ]*[а-яё-]*|
    с[- ]*[ао][- ]*д[- ]*[ао][- ]*м[- ]*[аи][- ]*[а-яё-]*|
    ч[- ]*и[- ]*н[- ]*а[- ]*р[- ]*[а-яё-]*|
    ч[- ]*[еи][- ]*р[- ]*к[- ]*а[- ]*ш[- ]*[а-яё-]*|
    ш[- ]*м[- ]*[оа][- ]*к[- ]*[а-яё-]*|
    ш[- ]*м[- ]*у[- ]*р[- ]*д[- ]*[а-яё-]*|
    н[- ]*а[- ]*р[- ]*и[- ]*к[- ]*[а-яё-]*|
    [а-яё-]*н[- ]*[- ]*а[- ]*ц[- ]*и[- ]*с[- ]*[а-яё-]*|
    ф[- ]*а[- ]*ш[- ]*и[- ]*с[- ]*т[- ]*[а-яё-]*|
    н[- ]*а[- ]*ц[- ]*и[- ]*к[- ]*[а-яё-]*|
    ф[- ]*а[- ]*ш[- ]*и[- ]*к[- ]*[а-яё-]*|
    к[- ]*[ао][- ]*м[- ]*м[- ]*у[- ]*н[- ]*я[- ]*к[- ]*[а-яё-]*|
    к[- ]*[ао][- ]*м[- ]*у[- ]*н[- ]*я[- ]*к[- ]*[а-яё-]*|
    л[- ]*[еи][- ]*б[- ]*[ие][- ]*р[- ]*а[- ]*[шс][- ]*[а-яё-]*|
    р[- ]*а[- ]*ш[- ]*и[- ]*с[- ]*т[- ]*[а-яё-]*|
    
)(?![а-яё])|

# 8. Другие
(?:х[- ]*а[- ]*м[- ]*[а-яё-]*|х[- ]*а[- ]*м[- ]*л[- ]*[ао][- ]*[а-яё-]*|м[- ]*р[- ]*а[- ]*з[- ]*[аоуий][- ]*|м[- ]*р[- ]*а[- ]*з[- ]*[ийь][- ]*|м[- ]*р[- ]*а[- ]*з[- ]*о[- ]*т[- ]*[а-яё-]*|а[- ]*у[- ]*т[- ]*[ий][- ]*с[- ]*т[- ]*[а-яё-]*|а[- ]*х[- ]*у[- ]*е[- ]*[а-яё-]*|д[- ]*е[- ]*г[- ]*е[- ]*н[- ]*е[- ]*р[- ]*а[- ]*т[- ]*[а-яё-]*|к[- ]*о[- ]*н[- ]*ч[- ]*[а-яё-]*)
)(?![а-яё])"""
import re
word = "профессиональной"
print(f"{word}: {bool(re.fullmatch(mat_regex, word, flags=re.VERBOSE))}")

профессиональной: False
